In [1]:
import pandas as pd 
import glob
import xarray as xr
import time

In [44]:
path = "data/114"

more_files = sorted(glob.glob(path + "/*.dat"))[:20] 
#prøverunnde med noen filer. denne finner alle filene som slutterpå ".dat" og leser bare de 10 første
#ikke utvit til noe særlig mer en typ 20-30 stasjoner. da sprenger pcen. burde gå bedre når vi effektiviserer etterpå


first = True
start = time.time()
for i in more_files:
    
    df = pd.read_table(i,skiprows=2, delim_whitespace=True,skipfooter=1,engine="python").drop(["%year","day"],axis=1)
    
    
    if(df.values.shape[0]<10):
        continue
    print(df.values.shape[0], int(str(meta.head().columns[3])[:-1]))
    
    meta = pd.read_table(i,skiprows=None,sep="\s+",nrows=1,engine="python")
    #finn en bedre måte å behandle de forskjellige delene av filene på. 
    #nå åpner jeg fila 2 ganer for å finne forskjellige deler av datan. jævlig teit
    measurement_time = pd.to_datetime(float(meta.values[0,1]),origin=str(int(meta.values[0,0])),unit="D")
    measurement_lat  = float(meta.values[0,2])
    measurement_lon  = float(meta.values[0,3])
    #print(measurement_time)
    
    df.rename(columns={"pressure(dbar)":"depth"}, inplace=True) # noget unøaktig konvertering eller?
    df = df.set_index("depth")
    

    ds = xr.Dataset.from_dataframe(df)
    
    #standard names
    ds = ds.rename({"salinity":"sea_water_salinity","temperature(C)":"sea_water_temperature"})

    ds["time"] = measurement_time
    ds["lat"]  = measurement_lat
    ds["lon"]  = measurement_lon
    
    #profile_nr = int(list(pd.read_table(i,sep="[:, ]",nrows=0,engine="python"))[4])
    profile_nr = int(str(meta.head().columns[3])[:-1])
    
    
    ds = ds.assign_coords(profile=profile_nr)
    ds = ds.expand_dims("profile")

    
    # joining files
    if first==True:
        itp114 = ds
        first=False
    else:
        itp114=xr.concat([itp114,ds],dim = "profile")
        #concat gjør at det blir mye nan i temp og salinity, men det følger cf. hør med Steingod

print("Det tok", time.time()-start)

#La oss fikse litt metadata da
#legg til alle de her greiene: https://adc.met.no/node/4

ds.time.attrs["standard_name"] = "time"
ds.time.attrs["units"] = "seconds since 1970-01-01 00:00:00+0"# husk å faktisk endre til dette formatet
ds["sea_water_temperature"].attrs["standard_name"] = "sea_water_temperature"
ds["sea_water_temperature"].attrs["units"] = "celcius" # husk å bytte til kelvin

ds.attrs["featureType"] = "trajectoryProfile"

itp114

233 19
378 0
120 1
122 3
119 4
121 8
120 9
121 13
119 14
121 18
Det tok 0.1915576457977295


<xarray.Dataset>
Dimensions:                (depth: 378, profile: 10)
Coordinates:
  * depth                  (depth) int64 6 8 10 12 14 16 ... 752 754 756 758 760
  * profile                (profile) int64 0 1 3 4 8 9 13 14 18 19
Data variables:
    sea_water_temperature  (profile, depth) float64 nan nan nan ... nan nan nan
    sea_water_salinity     (profile, depth) float64 nan nan nan ... nan nan nan
    time                   (profile) datetime64[ns] 2019-09-22T02:02:04.128000 ... 2019-09-24T00:02:00.960000
    lat                    (profile) float64 -145.2 -145.9 ... -146.3 -146.3
    lon                    (profile) float64 77.97 77.99 77.99 ... 78.0 77.99

Info:
====
**infoen er forløpig basert på gjennomgang av bare en bøye + at jeg har sett litt på hva som står om de andre. Så den kan være ugenerell, oppdaterer fortløpende. **


Data fra bøyene inholder profiler med trykk, temp og salt. 
Hver profil har en .dat fil med en header som inneholder starttid for profilen, lat,long og hvor mange målinger fila inneholder. enhet for tid er ÅR DAG, der dag har desimaler som gir klokkeslett. dag begynner på 1, ikke null. 

Noen bøyer har også andre målinger: bøya jeg nå ser på, 114, har to fastmonterte microcatsensorer som måler på 5 og 6 meters dyp hvert kvarter. de ligger i filene som har micro i navnet og er mye større. 

Jobber i første omgang med å lage et program som henter data fra én profil og en bøye.


Lage et program som dumper dette til netCDF-CF. 


Tanker og Spørsmål:
============



Ser ut til at metadata og discover-metadata må hardcodes for å stemme med standarene for [metadata]() og [discover-metadata](https://adc.met.no/node/4)

Skal jeg bruke trajectoryProfile eller Profile

skal jeg inkludere mlinger fra f.eks microcat-sensorene 